# Fake news detection problem

In [1]:
# importing the library

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [3]:
# Load data files 
df_fake = pd.read_csv('Fake.csv')
df_true = pd.read_csv('True.csv')

In [4]:
df_fake.head(2)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"


In [5]:
df_true.head(2)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"


In [6]:
df_fake['class'] = 0
df_true['class'] = 1

In [7]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [8]:
# remove last 10 rows 
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470, -1):
    df_fake.drop([i], axis=0, inplace=True)
df_true_manual_testing = df_true.tail(10)
for i in range(21416, 21406, -1):
    df_true.drop([i], axis=0, inplace=True)

In [9]:
# file shape
df_fake.shape, df_true.shape

((23471, 5), (21407, 5))

In [10]:
# New class create 
df_fake_manual_testing['class'] = 0
df_true_manual_testing['class'] = 1

/tmp/ipykernel_199577/1523065411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing['class'] = 0
/tmp/ipykernel_199577/1523065411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing['class'] = 1


In [11]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [12]:
df_manual_testing = pd.concat([df_fake_manual_testing, 
                               df_true_manual_testing], axis=0)
df_manual_testing.to_csv('manual_testing.csv')

Merging True and fake Dataframes

In [13]:
df_merge = pd.concat([df_fake, df_true], axis=0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [14]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [15]:
df = df_merge.drop(['title','subject','date'], axis=1)
df.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [16]:
# suffle the dataset
df = df.sample(frac=1)

In [17]:
df.head(10)

,text,class
8326,WASHINGTON (Reuters) - Senator John McCain sur...,1
20454,Read anything Stephen Henderson has written ov...,0
13587,Don t you just love all the lies coming out of...,0
13109,The Left is not able to get away with shaming ...,0
638,"Trump hasn t made the economy better, no matte...",0
17370,SEOUL (Reuters) - Propaganda fliers presumed t...,1
17806,The View s Joy Behar was beaming after watchin...,0
11585,WASHINGTON (Reuters) - Russia is likely to mai...,1
12779,,0
7586,Wednesday night was the final Democratic deba...,0


In [18]:
# check nul values in our datasets 
df.isnull().sum()

text     0
class    0
dtype: int64

Randomly Shuffling The dataframe

In [19]:
df = df.sample(frac=1)

In [20]:
df.head()

,text,class
3704,"Recently, news broke that the Trump Foundation...",0
10046,(Reuters) - The Tennessee House passed a bill ...,1
4638,WASHINGTON (Reuters) - A Republican-controlled...,1
2086,ISLAMABAD (Reuters) - Pakistan has taken actio...,1
269,Like increasing numbers of heartless people on...,0


In [21]:
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)

In [22]:
df.columns

Index(['text', 'class'], dtype='object')

In [23]:
df.head()

,text,class
0,"Recently, news broke that the Trump Foundation...",0
1,(Reuters) - The Tennessee House passed a bill ...,1
2,WASHINGTON (Reuters) - A Republican-controlled...,1
3,ISLAMABAD (Reuters) - Pakistan has taken actio...,1
4,Like increasing numbers of heartless people on...,0


Creating a function to convert the text in lowercase, remove the extra
space, special chr.., ulr and links

In [24]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text
    

In [25]:
df['text'] = df['text'].apply(wordopt)

Defining and dependent and independent variable as x and y

In [26]:
x = df['text']
y = df['class']

Splitting the dataset into training set and and testing set 

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

convert text to vectors 

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

1.Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [32]:
pred_lr = LR.predict(xv_test)
LR.score(xv_test, y_test)

0.9879679144385026

In [33]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5817
           1       0.99      0.99      0.99      5403

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



2.Decision Tree Classification

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [36]:
pred_dt = DT.predict(xv_test)

In [37]:
DT.score(xv_test, y_test)

0.9951871657754011

In [38]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5817
           1       1.00      0.99      1.00      5403

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [39]:
from sklearn.ensemble import GradientBoostingClassifier

In [40]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [41]:
pred_gbc = GBC.predict(xv_test)

In [42]:
GBC.score(xv_test, y_test)

0.9950089126559715

In [43]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5817
           1       0.99      1.00      0.99      5403

    accuracy                           1.00     11220
   macro avg       0.99      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



4. Random Forest Classifier

In [44]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [46]:
pred_rfc = RFC.predict(xv_test)

In [47]:
RFC.score(xv_test, y_test)

0.990909090909091

In [48]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5817
           1       0.99      0.99      0.99      5403

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



5. SVM 

Model Testing with manual entry

In [49]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0]))) 
                                                                       

# Predict News Manually

In [50]:
news = str(input())
manual_testing(news)

 fata




LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News
